In [1]:
import os
import shutil
import subprocess

# Make Kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Move API key to the correct location
shutil.copy('/content/kaggle.json', '/root/.kaggle/kaggle.json')
os.chmod('/root/.kaggle/kaggle.json', 0o600)

# Install Kaggle CLI
subprocess.run(['pip', 'install', 'kaggle'])

# Download the dataset
subprocess.run([
    'kaggle', 'datasets', 'download',
    '-d', 'paultimothymooney/chest-xray-pneumonia',
    '--force'
])

# Unzip the dataset
import zipfile
with zipfile.ZipFile('chest-xray-pneumonia.zip', 'r') as z:
    z.extractall()


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'chest_xray/train', target_size=(224,224), batch_size=32, class_mode='binary')

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'chest_xray/val', target_size=(224,224), batch_size=32, class_mode='binary')

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    'chest_xray/test', target_size=(224,224), batch_size=32, class_mode='binary')


Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [3]:
base = tf.keras.applications.MobileNetV2(
    input_shape=(224,224,3), include_top=False, weights='imagenet'
)
base.trainable = False

model = tf.keras.Sequential([
    base,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train (for demo, just 3 epochs; increase as needed for better results)
history = model.fit(train_gen, validation_data=val_gen, epochs=3)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 66s 342ms/step - accuracy: 0.8400 - loss: 0.3613 - val_accuracy: 0.6875 - val_loss: 0.5109
Epoch 2/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 55s 338ms/step - accuracy: 0.9560 - loss: 0.1391 - val_accuracy: 0.8750 - val_loss: 0.3408
Epoch 3/3
163/163 ━━━━━━━━━━━━━━━━━━━━ 50s 305ms/step - accuracy: 0.9638 - loss: 0.1092 - val_accuracy: 0.8750 - val_loss: 0.3504


In [4]:
loss, acc = model.evaluate(test_gen)
print(f"Test accuracy: {acc:.2%}")


20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 233ms/step - accuracy: 0.8311 - loss: 0.3813
Test accuracy: 83.81%


In [5]:
model.save('cxr_pneumonia_model.keras')
